In [3]:
import requests
import json
from datetime import datetime
import uuid

class ApiClient:
    def __init__(self, config):
        self.base_url = f"http://{config['ip']}:9999"
        self.api_source = config['api_source']
        self.database = config['dbSource']
        self.headers = {
            "Accept": "*/*",
            "User-Agent": "Thunder Client (https://www.thunderclient.com)",
            "Content-Type": "application/json"
        }

    def create_schema(self):
        data = {
            "dbName": self.database["dbName"],
            "schemaName": self.database["schemaName"],
            "host": self.database["host"],
            "port": self.database["port"],
            "user": self.database["user"]
        }
        payload = json.dumps(data)
        response = requests.post(f"{self.base_url}/create_schema", data=payload, headers=self.headers)
        return response.text
    
    def create_vpc(self):
        data = {
            "apiSource": self.api_source,
            "dbSource": {
                "dbName": self.database["dbName"],
                "schemaName": self.database["schemaName"],
                "schemaPath": "../schema/naverCloudSchema.sql",
                "host": self.database["host"],
                "port": self.database["port"],
                "user": self.database["user"]
            }
        }
        payload = json.dumps(data)
        response = requests.post(f"{self.base_url}/create_vpc", data=payload, headers=self.headers)
        return response.text
    
    def create_recovery(self):
        response = requests.post(f"{self.base_url}/create_recovery", data={}, headers=self.headers)
        return response.text
    
    def set_resource_info(self, data):
        # payload = json.dumps(data)
        payload = data
        response = requests.post(f"{self.base_url}/set_resource_info", data=payload, headers=self.headers)
        return response.text

    def set_recovery_info(self, data):
        payload = json.dumps(data)
        # payload = data
        response = requests.post(f"{self.base_url}/set_recovery_info", data=payload, headers=self.headers)
        return response.text
    
    def source_to_target(self):
        data = {}
        payload = json.dumps(data)
        response = requests.post(f"{self.base_url}/source_to_target", data={}, headers=self.headers)
        return response.text
        
    def get_resource_list(self):
        data = {}
        payload = json.dumps(data)
        response = requests.post(f"{self.base_url}/get_resource_list", data=payload, headers=self.headers)
        return response.text

    def sync_cluster(self):
        data = {}
        payload = json.dumps(data)
        response = requests.post(f"{self.base_url}/sync_cluster", data=payload, headers=self.headers)
        return response.text
    
    def read2insert(self):
        data = {
            "apiSource": self.api_source,
            "dbSource": {
                "dbName": self.database["dbName"],
                "schemaName": self.database["schemaName"],
                "schemaPath": "../schema/naverCloudSchema.sql",
                "host": self.database["host"],
                "port": self.database["port"],
                "user": self.database["user"]
            }
        }
        payload = json.dumps(data)
        response = requests.post(f"{self.base_url}/read2insert", data=payload, headers=self.headers)
        return response.text

    def recovery_vpc(self):
        if self.api_source is None or self.database is None:
            return "API source or database configuration is missing."
        
        data = {
            "apiSource": self.api_source,
            "dbSource": {
                "dbName": self.database["dbName"],
                "schemaName": self.database["schemaName"],
                "schemaPath": "../schema/naverCloudSchema.sql",
                "host": self.database["host"],
                "port": self.database["port"],
                "user": self.database["user"]
            }
        }
        payload = json.dumps(data)
        response = requests.post(f"{self.base_url}/recovery_vpc", data=payload, headers=self.headers)
        return response.text

In [4]:
import connDbnApi as cda

dbSource = {
    'dbName': 'cdm_fix', 
    'schemaName': 'test240925t',
    'host': '175.45.214.45',
    'port': '26257',
    'user': 'root'
}


conn = cda.Connect(db = dbSource).connect_cockroachdb()
conn.autocommit = True
cur = conn.cursor()
with open('../schema/naverCloudSchema.sql', 'r', encoding='UTF8') as file:
    sql = file.read()
sql = sql.replace('CREATE TABLE IF NOT EXISTS', f'CREATE TABLE IF NOT EXISTS "rs_1735540888798".')
sql = sql.replace('CREATE SEQUENCE IF NOT EXISTS ', f'CREATE SEQUENCE IF NOT EXISTS "rs_1735540888798".')
cur.execute(f'set schema rs_1735540888798;')
# cur.execute(sql)
cur.close()
conn.close()

In [5]:
conn = cda.Connect(db = dbSource).connect_cockroachdb()
cur = conn.cursor()

cur.execute(f"use cdm_fix;")

# cur.execute(q)

In [6]:
#src
config1 = {
    'ip': '10.255.93.237', # API Server IPaddr
    'api_source': {
        'accessKey': 'mYUP1ZqESUOpjyOokWC8',
        'secretKey': '31scunD8FAtSTqU92X2DYFsi1UaiEbQ5qrTxi2aM',
        'ncloudUrl': 'https://ncloud.apigw.gov-ntruss.com',
        'billingApiUrl': 'https://billingapi.apigw.gov-ntruss.com'
    },
    'dbSource': {
        'dbName': 'cdm_fix', # API Server IPaddr
        'schemaName': 'test240925t',
        'host': '175.45.214.45',
        'port': '26257',
        'user': 'root'
    }
}
#tgt
config2 = {
    'ip': '10.255.93.237',
    'api_source': {
        'accessKey': '9AB413B39F22F35B57BD',
        'secretKey': 'E2B9D2670DCB51B93A595E75D21140771105C1E0',
        'ncloudUrl': 'https://ncloud.apigw.gov-ntruss.com',
        'billingApiUrl': 'https://billingapi.apigw.gov-ntruss.com'
    },
    'dbSource': {
        'dbName': 'cdm_fix',
        'schemaName': 'test240925t',
        'host': '175.45.214.45',
        'port': '26257',
        'user': 'root'
    }
}
src_client = ApiClient(config1)
tgt_client = ApiClient(config2)

def transform_to_request_ex(initial_data):
    """
    초기 데이터를 받아서 request_ex 형식으로 변환합니다.
    
    Args:
        initial_data (dict): 변환할 초기 데이터.
        
    Returns:
        dict: 변환된 request_ex 데이터.
    """
    
    # 고유한 요청 ID 생성
    request_id = str(uuid.uuid4())
    
    # 현재 UTC 시간 ISO 형식으로 가져오기
    current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # 'Z'는 UTC 시간을 나타냅니다.
    
    # 초기 데이터에서 필요한 필드 추출
    plan = initial_data.get("parameter", {}).get("data", {}).get("plan", {})
    plan_id = plan.get("id")
    plan_name = plan.get("name")
    instances = plan.get("instance", [])
    
    # 인스턴스가 존재하는지 확인하고 첫 번째 인스턴스 사용
    if instances:
        instance = instances[0]
        instance_uuid = instance.get("uuid")
        instance_name = instance.get("name")
    else:
        instance_uuid = None
        instance_name = None
    
    # 변환된 request_ex 구조 생성
    request_ex = {
        "request": {
            "id": request_id,  # 고유 ID
            "code": initial_data.get("code"),
            "parameter": {
                "command": initial_data.get("parameter", {}).get("command"),
                "data": {
                    "plan": {
                        "id": plan_id,
                        "name": plan_name,
                        "raw": {
                            "requestid": plan_id,
                            "requestname": plan_name,
                            "resourcetype": "serverinstance",
                            "sourcekey": instance_uuid,
                            "timestamp": current_timestamp,
                            "command": "CREATE",
                            "detail": instance_name,
                            "completeflag": False
                        }
                    }
                }
            }
        }
    }
    
    return request_ex



In [13]:
# Sync Cluster
sync_ex = {
"code": "clusterinfo",
    "parameter" : {
        "command": "sync",
        "data": {
        }
    }
}
command_type = sync_ex["parameter"]["command"]
if command_type == "sync":
    print(src_client.sync_cluster())

# Cluster Info registered -> If got Sync sign -> ... -> when get rsc info set, send 

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
  "http://www.w3.org/TR/html4/loose.dtd">
<html>
  <head>
    <title>psycopg2.errors.InvalidSchemaName: cannot create &quot;rs_1735542054891.accesscontrolgroup_seq&quot; because the target database or schema does not exist
HINT:  verify that the current database and search_path are valid and/or the target database exists // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"
        type="text/css">
    <!-- We need to make sure this has a favicon so that the debugger does
         not accidentally trigger a request to /favicon.ico which might
         change the application's state. -->
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script type="text/javascript">
      var TRACEBACK = 1427536594928,
          CONSOLE_MODE = false

In [13]:
#step 1 : got request and set recovery plan

initial_data = {
    "code": "recoveryinfo",
    "parameter": {
        "command": "set",
        "data": {
            "plan": {
                "id": 1,
                "name": "target-A",
                "instance": [
                    {
                        "uuid": "3081226",
                        "name": "servername"
                    }
                ]
            }
        }
    }
}

res_req = transform_to_request_ex(initial_data)

formatted_req = res_req["request"]["parameter"]["data"]["plan"]["raw"]
command_type = res_req["request"]["parameter"]["command"]
print("formatted_req: ", formatted_req)
src_client.set_recovery_info(formatted_req)
print("recovery plan set")

formatted_req:  {'requestid': 1, 'requestname': 'target-A', 'resourcetype': 'serverinstance', 'sourcekey': '3081226', 'timestamp': '2024-12-30 17:28:59', 'command': 'CREATE', 'detail': 'servername', 'completeflag': False}
recovery plan set


In [14]:
# step 2: run recovery plan
request_ex_2 = {
  "code": "recoveryjob",
    "parameter": {
       "command": "run",
       "data": {
            "plan": {
                "id": 44,
                "name": "500",
                "job_id": 100
            }
       }
    }
}
# def generate_recovery_info(requestid, resourcetype, sourcekey, command, detail=None, completeflag=False):
#     recovery_info = {
#         'requestid': requestid,
#         'resourcetype': resourcetype,
#         'sourcekey': sourcekey,
#         'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
#         'command': command,
#         'detail': detail,
#         'completeflag': completeflag
#     }
#     return recovery_info

# recovery_info = generate_recovery_info(
#     requestid='44',
#     resourcetype='serverinstance',
#     sourcekey='3081226',#target-contoller key
#     command='CREATE',
#     detail="servername",
#     completeflag=False
# )

# src_client.set_recovery_info(recovery_info)

command_type = request_ex_2["parameter"]["command"]
if command_type == "run":
    # Run recovery plan
    print("recovery plan run")
    res = src_client.create_vpc()


recovery plan run


In [12]:
# res
import pickle
with open("history.pkl","rb") as fr:
    data = pickle.load(fr)
print(data)

{'677254b2cce3516444ea8886': {'historyId': '677254b2cce3516444ea8886', 'nrn': 'nrn:GOV:VPCServer:KR:9756:NetworkInterface/456379', 'eventTime': 1735546034474, 'platformType': 'VPC', 'productName': 'VPCServer', 'productDisplayName': 'Server', 'regionCode': 'KR', 'regionDisplayName': '수도권', 'resourceType': 'NetworkInterface', 'resourceId': '456379', 'resourceName': 'nic-456379', 'actionDisplayName': 'Attach Network Interface (Complete)', 'actionResultType': 'SUCCESS', 'actionUserType': 'Customer', 'sourceType': 'SYSTEM', 'sourceIp': '203.255.93.2', 'productData': {'ipAllotementBlockNo': '1', 'defaultYn': 'Y', 'createdYmdt': '1735546001045', 'vpcName': 'netdb', 'overlayIp': '10.0.0.10', 'subnetNo': '19238', 'instanceTypeCode': 'SVR', 'networkInterfaceNo': '456379', 'networkInterfaceName': 'nic-456379', 'regionNo': '1', 'deleteOnTerminationYn': 'Y', 'memberNo': '9756', 'macAddress': 'F2:20:EF:44:77:F5', 'zoneNo': '2', 'modifiedYmdt': '1735546034474', 'vpcNo': '5119753', 'statusCode': 'USED

In [33]:
# request_ex = {
#   "request": {
#     "id": "ae9f803c-044e-4995-b561-b342e136597a",
#     "code": "recoveryinfo",
#     "parameter": {
#       "command": "set",
#       "data": {
#         "plan": {
#           "id": "406",
#           "name": "dr_name",
#           "raw": {
#             "requestid": "406",
#             "requestname": "dr_name",
#             "resourcetype": "serverinstance",
#             "sourcekey": "3081226",
#             "timestamp": "2024-10-01 11:02:33",
#             "command": "CREATE",
#             "detail": None,
#             "completeflag": False
#           }
#         }
#       }
#     }
#   }
# }



# request_ex = {
#   "request": {
#     "code": "recoveryinfo",
#     "parameter": {
#         "command": "set",
#         "data": {
#             "plan": {
#                 "id": 1,
#                 "name": "s18fb360bc62 ",
#                 "instance": [
#                     {
#                         "uuid": "3051719",
#                         "name": "s18fb360bc62"
#                     }
#                 ]
#             }
#         }
#     }
#   }
# }

# request_ex2 = {
#   "request": {
#     "code": "recoveryinfo",
#     "parameter": {
#         "command": "set",
#         "data": {
#             "plan": {
#                 "id": 1,
#                 "name": "vs19222b1c528",
#                 "instance": [
#                     {
#                         "uuid": "1700975",
#                         "name": "vs19222b1c528"
#                     }
#                 ]
#             }
#         }
#     }
#   }
# }

# uuid = request_ex["request"]["parameter"]["data"]["plan"]["instance"]["uuid"]
# command_type = request_ex["request"]["parameter"]["command"]
# query_to_make_req = f"SELECT * FROM {config1.source_db['schemaName']}.{self.table_name}"
# formatted_req = 
# src_client.set_recovery_info(formatted_req)

# print(res_req)
# print(request_ex)